<a href="https://colab.research.google.com/github/skabra5/Streaming-tweets-with-Tweepy-Visualing-tweets-using-Plotly-real-time-topic-tracking-with-Dash/blob/master/DashApp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
os.chdir("/content/drive/My Drive/Colab Notebooks/TwitterDash")

In [1]:
!pip install dash dash-renderer dash-html-components dash-core-components plotly

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input, State
import pandas as pd
import plotly.graph_objs as go
from django.conf import settings
import itertools
import math
import base64
from flask import Flask
import os
import psycopg2
import datetime
import plotly
import random
from collections import deque
 
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob
import sqlite3



/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning:

The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [1]:
### Install ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

### Run ngrok to tunnel Dash app port 8050 to the outside world. 
### This command runs in the background.
get_ipython().system_raw('./ngrok http 8050 &')

### Get the public URL where you can access the Dash app. Copy this URL.
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


--2020-06-10 07:04:43--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.201.248.214, 35.175.20.97, 34.238.5.126, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.201.248.214|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  17.9MB/s    in 0.7s    

2020-06-10 07:04:44 (17.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
http://281021dc10ad.ngrok.io


In [2]:
%%writefile app.py
import dash
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html
import plotly
import random
import plotly.graph_objs as go
from collections import deque
import sqlite3
import pandas as pd
import time
import os

#popular topics: google, olympics, trump, gun, usa


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.scripts.config.serve_locally = True
app.title = "Real-time-twitter-monitor"

app.layout = html.Div(
    [   html.H2('Live Twitter Sentiment'),
        dcc.Input(id='sentiment_term', value='olympic', type='text'),
        dcc.Graph(id='live-update-graph', animate=False),
        dcc.Interval(
            id='interval-component-slow',
            interval=1*1000,
            n_intervals=0
        ),

    ]
)

@app.callback(Output('live-update-graph', 'figure'), [Input('sentiment_term',"value"), Input('interval-component-slow', 'n_intervals')])
def update_graph_scatter(sentiment_term):
  try:
    conn = sqlite3.connect('/content/twitter.db')
    c = conn.cursor()
    df = pd.read_sql("SELECT * FROM sentiment WHERE tweet LIKE ? ORDER BY unix DESC LIMIT 1000", conn ,params=('%' + sentiment_term + '%',))
    df.sort_values('unix', inplace=True)
    df['sentiment_smoothed'] = df['polarity'].rolling(int(len(df)/2)).mean()

    df['date'] = pd.to_datetime(df['unix'],unit='ms')
    df.set_index('date', inplace=True)

    df = df.resample('1min').mean()
    df.dropna(inplace=True)

    X = df.unix.values[-100:]
    Y = df.sentiment_smoothed.values[-100:]

    data = plotly.graph_objs.Scatter(
            x=X,
            y=Y,
            name='Scatter',
            mode= 'lines+markers'
            )

    return {'data': [data],'layout' : go.Layout(xaxis=dict(range=[min(X),max(X)]),yaxis=dict(range=[min(Y),max(Y)]),title='Term: {}'.format(sentiment_term))}

  except Exception as e:
    with open('errors.txt','a') as f:
      f.write(str(e))
      f.write('\n')

if __name__ == '__main__':
    app.run_server(debug=True)

Overwriting app.py


In [3]:
!python app.py

Running on http://127.0.0.1:8050/
Debugger PIN: 137-148-803
 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
Running on http://127.0.0.1:8050/
Debugger PIN: 244-471-906
